In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
camera_list = [
    {'name': 'top', 'serial': 22181547},
    #{'name': 'side1', 'serial': 22181612},
]

In [3]:
import PySpin
from multicamera_acquisition.interfaces import get_camera
from datetime import datetime
import csv
from multicamera_acquisition.paths import ensure_dir
from multicamera_acquisition.acquisition import Writer, AcquisitionLoop
import serial
from pathlib2 import Path
from datetime import datetime
import multiprocessing as mp
import warnings

/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from multicamera_acquisition.paths import DATA_DIR
import logging
from multicamera_acquisition.video_utils import count_frames

In [5]:
brand = 'flir'
save_location = DATA_DIR / 'tests'
framerate = 30
exposure_time = 2000
recording_duration_s = 10
serial_timeout_duration_s=0.1
overwrite=False
append_datetime=True
verbose = True
n_input_trigger_states =4 

In [6]:
if brand not in ['flir', 'basler']:
    raise NotImplementedError

# get Path of save location
if type(save_location) != Path:
    save_location = Path(save_location)

# create a subfolder for the current datetime
if append_datetime:
    date_str = datetime.now().strftime("%y-%m-%d-%H-%M-%S-%f")
    save_location = save_location / date_str

# ensure that a directory exists to save data in
ensure_dir(save_location)

triggerdata_file = save_location / "triggerdata.csv"
if triggerdata_file.exists() and (overwrite == False):
    raise FileExistsError(f"CSV file {triggerdata_file} already exists")

In [7]:
# initialize cameras
writers = []
acquisition_loops = []

In [8]:
# create acquisition loops
for camera_dict in camera_list:
    break

In [9]:
name = camera_dict["name"]
serial_number = camera_dict["serial"]

video_file = save_location / f"{name}.{serial_number}.avi"
metadata_file = save_location / f"{name}.{serial_number}.triggerdata.csv"

if video_file.exists() and (overwrite == False):
    raise FileExistsError(f"Video file {video_file} already exists")

In [10]:
# create a writer queue
write_queue = mp.Queue()
writer = Writer(
    write_queue,
    video_file_name=video_file,
    metadata_file_name=metadata_file,
    fps=framerate,
    camera_serial=serial_number,
    camera_name=name,
)

In [11]:
# prepare the acuqisition loop in a separate thread
acquisition_loop = AcquisitionLoop(
    write_queue, brand = brand, serial_number=serial_number, exposure_time=exposure_time, gain=15
)

In [12]:
# initialize acquisition
writer.start()
writers.append(writer)
acquisition_loop.start()
acquisition_loop.ready.wait()
acquisition_loops.append(acquisition_loop)
if verbose:
    logging.info(f"Initialized {name} ({serial_number})")

/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:69: UserWarning: Dropped 1 frame on #22181547: 
SpinnakerException
  warnings.warn(
/home/dattalab/code/multicamera_acquisition/multicamera_acquisition/acquisition.py:69: UserWarning: Dropped 152 frame on #22181547: 
SpinnakerException
  warnings.warn(


In [13]:
# prepare acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.prime()
    acquisition_loop.ready.wait()

In [14]:
import glob

In [15]:
# prepare communication with arduino
serial_ports = glob.glob("/dev/ttyACM*")
# check that there is an arduino available
if len(serial_ports) == 0:
    raise ValueError("No serial device (i.e. Arduino) available to capture frames")
port = glob.glob("/dev/ttyACM*")[0]
arduino = serial.Serial(port=port, timeout=serial_timeout_duration_s)

In [16]:
import time

In [17]:
# delay recording to allow serial connection to connect
time.sleep(1.0)

# create a triggerdata file
with open(triggerdata_file, "w") as triggerdata_f:
    triggerdata_writer = csv.writer(triggerdata_f)
    triggerdata_writer.writerow(
        ["pulse_id", "arduino_ms"]
        + [f"flag_{i}" for i in range(n_input_trigger_states)]
    )

In [18]:
from multicamera_acquisition.interfaces.arduino import packIntAsLong, wait_for_serial_confirmation

In [19]:
from tqdm.autonotebook import tqdm

In [20]:
wait_for_serial_confirmation

<function multicamera_acquisition.interfaces.arduino.wait_for_serial_confirmation(arduino, expected_confirmation, recording_duration=5, timeout_duration_s=0.1)>

In [21]:
# Tell the arduino to start recording by sending along the recording parameters
inv_framerate = int(1e6 / framerate)
num_cycles = int(recording_duration_s * framerate / 2)
msg = b"".join(
    map(
        packIntAsLong,
        (
            num_cycles,
            exposure_time,
            inv_framerate,
        ),
    )
)
arduino.write(msg)

12

In [22]:
from datetime import timedelta

In [23]:
# Run acquision
confirmation = wait_for_serial_confirmation(arduino, "Start")
 # how long to record
endtime = datetime.now() + timedelta(seconds=recording_duration_s+10) 
# while current time is less than initial time + recording_duration_s
while datetime.now() < endtime:
    confirmation = arduino.readline().decode("utf-8").strip("\r\n")

    # save input data flags
    if len(confirmation) > 0:
        print(confirmation)
        if confirmation[:7] == "input: ":
            with open(triggerdata_file, "a") as triggerdata_f:
                triggerdata_writer = csv.writer(triggerdata_f)
                states = confirmation[7:].split(",")[:-2]
                frame_num = confirmation[7:].split(",")[-2]
                arduino_clock = confirmation[7:].split(",")[-1]
                triggerdata_writer.writerow([frame_num, arduino_clock] + states)
        if verbose:
            print(confirmation)

    if confirmation == "Finished":
        print('End Acquisition')
        break

if confirmation != "Finished":
    confirmation = wait_for_serial_confirmation(arduino, "Finished")

Confirmation recieved: Start
input: 1,0,0,0,1,709
input: 1,0,0,0,1,709
Finished
Finished
End Acquisition


In [24]:
from datetime import datetime, timedelta

In [25]:
datetime.now()+ timedelta(seconds=recording_duration_s) > datetime.now()

True

In [26]:
# end acquisition loops
for acquisition_loop in acquisition_loops:
    acquisition_loop.stop()
    acquisition_loop.join()

In [27]:
# @CALEB: what is the purpose of this?
for writer in writers:
    writer.join()

In [28]:
video_file

PosixPath('/home/dattalab/code/multicamera_acquisition/data/tests/23-01-12-12-15-48-151327/top.22181547.avi')

In [29]:
if verbose:
    # count each frame
    for camera_dict in camera_list:
        name = camera_dict["name"]
        serial_number = camera_dict["serial"]
        video_file = save_location / f"{name}.{serial_number}.avi"
        print(f"Frames ({name}):", count_frames(video_file.as_posix()))

Frames (top): 151
